In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load dataset
df = pd.read_csv("data/dirty_cafe_sales.csv")

In [3]:
# Creating a copy for a workable dataset
new_df = df

In [4]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [5]:
new_df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [6]:
new_df.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1961373,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


In [7]:
new_df.duplicated().sum()

0

In [8]:
# Replacing the strinf error value with NaN(easier to manupulate dataset)
new_df.replace(["UNKNOWN","ERROR"], np.nan, inplace=True)

In [9]:
# Converts columns to number and date
new_df["Transaction Date"] = pd.to_datetime(new_df["Transaction Date"], errors="coerce")
new_df[["Quantity","Price Per Unit","Total Spent"]] = new_df[["Quantity","Price Per Unit","Total Spent"]].apply(pd.to_numeric, errors="coerce")
# Forward-fill missing date
new_df["Transaction Date"] = new_df["Transaction Date"].ffill()

In [10]:
# Forward-fill missing Payment Mentod and Location
new_df["Payment Method"].ffill(inplace=True)
new_df["Location"].ffill(inplace=True)

C:\Users\gaura\AppData\Local\Temp\ipykernel_5044\4030765483.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  new_df["Payment Method"].ffill(inplace=True)
C:\Users\gaura\AppData\Local\Temp\ipykernel_5044\4030765483.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [11]:
# Calculate the missing value of Total Spent, Price Per Unit and Quantity
new_df["Total Spent"].fillna(new_df["Quantity"] * new_df["Price Per Unit"], inplace=True)
new_df["Price Per Unit"].fillna(new_df["Total Spent"] / new_df["Quantity"], inplace=True)
new_df["Quantity"].fillna(new_df["Total Spent"] / new_df["Price Per Unit"], inplace=True)

C:\Users\gaura\AppData\Local\Temp\ipykernel_5044\2650646029.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  new_df["Total Spent"].fillna(new_df["Quantity"] * new_df["Price Per Unit"], inplace=True)
C:\Users\gaura\AppData\Local\Temp\ipykernel_5044\2650646029.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting va

In [12]:
# Fill missing Price Per Unit using the most common price per item
item_price_map = new_df.groupby("Item")["Price Per Unit"].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
new_df["Price Per Unit"].fillna(new_df["Item"].map(item_price_map), inplace=True)

C:\Users\gaura\AppData\Local\Temp\ipykernel_5044\3782561165.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  new_df["Price Per Unit"].fillna(new_df["Item"].map(item_price_map), inplace=True)


In [13]:
# Filling missing Items names using most common items per price
price_item_map = new_df.groupby("Price Per Unit")["Item"].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
new_df["Item"].fillna(new_df["Price Per Unit"].map(price_item_map), inplace=True)

C:\Users\gaura\AppData\Local\Temp\ipykernel_5044\172224289.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  new_df["Item"].fillna(new_df["Price Per Unit"].map(price_item_map), inplace=True)


In [14]:
new_df.dropna(subset=["Total Spent","Quantity"], inplace=True)

In [15]:
new_df.isnull().sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

In [16]:
new_df.to_csv("output/cleaned_data.csv", index=False)

In [17]:
new_df

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,Credit Card,In-store,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2.0,2.0,4.0,Cash,In-store,2023-08-30
9996,TXN_9659401,Cookie,3.0,1.0,3.0,Digital Wallet,In-store,2023-06-02
9997,TXN_5255387,Coffee,4.0,2.0,8.0,Digital Wallet,In-store,2023-03-02
9998,TXN_7695629,Cookie,3.0,1.0,3.0,Digital Wallet,In-store,2023-12-02
